# Making the trainig sample from CANDELS GOODS-S 

By Shooby, last edited May29 2019



In [74]:
import numpy as np
import astropy.io.fits as pyfits
import matplotlib.pyplot as plt
import astropy.wcs as wcs
from astropy import units as u
from astropy.coordinates import SkyCoord
from scipy import ndimage
import scipy.misc


%matplotlib inline

In [2]:
gs = pyfits.getdata('../../WFIRST_WPS/CANDELS_fits/gds.fits')

In [70]:
sel = (gs['zbest']>0.1)&(gs['zbest']<1.5)&(gs['CLASS_STAR']<0.85)&(gs['FWHM_IMAGE']>15)
print (len(gs[sel]))

1668


In [71]:
def radec2xy(ra,dec,wc):
    coords = SkyCoord(ra,dec, unit='deg')
    a=wcs.utils.skycoord_to_pixel(coords, wc, origin=0,mode=u'wcs')
    return a[0],a[1]
    
def cut(ra,dec,andaze,filename):
    '''gets coordinates of the galaxy and the filter to return a cutout
    (also called a postage stamp) of the galaxy with given size'''
    hdr = pyfits.getheader(filename)
    w = wcs.WCS(hdr)
    x,y=radec2xy(ra,dec,w)
    x,y=np.int(x),np.int(y)
    im=pyfits.getdata(filename)[y-andaze:y+andaze,x-andaze:x+andaze]
    return im

def segment(a1,above_mean=1.1,add_image_back=0.3,random_noise=0.2):
    '''Returns segmentation of an image by some filtering of lower level 
    noises, to restrict the fitting to the galaxy itself and mask the 
    surroundings'''
    im2=ndimage.gaussian_filter(a1, 2)
    mask = (im2 > above_mean*im2.mean()).astype(np.float)
    mask += add_image_back * im2
    img = mask + random_noise*np.random.randn(*mask.shape)
    hist, bin_edges = np.histogram(img, bins=60)
    bin_centers = 0.5*(bin_edges[:-1] + bin_edges[1:])
    binary_im = img > random_noise
    open_img = ndimage.binary_opening(binary_im)
    close_img = ndimage.binary_closing(open_img)
    return close_img

In [77]:
ra,dec=gs['RA_1'][sel],gs['DEC_1'][sel]
im = np.zeros([50,50])
for boz in range(len(ra)):
    im = cut(ra[boz],dec[boz],25,'/Users/shemmati/Desktop/GOODS/goodss_all_wfc3_ir_f160w_060mas_v1.0_drz.fits')
    #plt.imshow(np.tanh(im),cmap='gray',origin='lower')
    #plt.colorbar()
    scipy.misc.imsave('images/'+str(boz)+'.jpg', im)
    #stop
    
    
    
    

/Users/shemmati/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  import sys
